In [ ]:
import pulp as pl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import numpy as np

class PackingProblemSolverPuLP:
    def __init__(self, N, L, W, H, l, w, h, wx, wy, wz):
        """
        Initialize the PuLP model for the 3D packing problem.
        """
        self.N = N
        self.L = L
        self.W = W
        self.H = H
        self.l = l
        self.w = w
        self.h = h
        self.wx = wx
        self.wy = wy
        self.wz = wz
        
        # Define the MILP model
        self.model = pl.LpProblem("3D_Packing_Problem", pl.LpMinimize)

        # Decision Variables
        self.x = [pl.LpVariable(f"x_{i}", lowBound=0, upBound=L, cat=pl.LpContinuous) for i in range(N)]
        self.y = [pl.LpVariable(f"y_{i}", lowBound=0, upBound=W, cat=pl.LpContinuous) for i in range(N)]
        self.z = [pl.LpVariable(f"z_{i}", lowBound=0, upBound=H, cat=pl.LpContinuous) for i in range(N)]

        # Binary variables for orientation (6 possible orientations)
        self.d = [[pl.LpVariable(f"d_{i}_{j}", cat=pl.LpBinary) for j in range(6)] for i in range(N)]

        # Binary variables for spatial relationships
        self.s = [[pl.LpVariable(f"s_{i}_{j}", cat=pl.LpBinary) for j in range(N)] for i in range(N)]
        self.u = [[pl.LpVariable(f"u_{i}_{j}", cat=pl.LpBinary) for j in range(N)] for i in range(N)]
        self.b = [[pl.LpVariable(f"b_{i}_{j}", cat=pl.LpBinary) for j in range(N)] for i in range(N)]

        # Transformed dimensions
        self.l_hat = [pl.LpVariable(f"l_hat_{i}", lowBound=0, cat=pl.LpContinuous) for i in range(N)]
        self.w_hat = [pl.LpVariable(f"w_hat_{i}", lowBound=0, cat=pl.LpContinuous) for i in range(N)]
        self.h_hat = [pl.LpVariable(f"h_hat_{i}", lowBound=0, cat=pl.LpContinuous) for i in range(N)]

    def add_constraints(self):
        """
        Add all constraints to the PuLP model.
        """
        for i in range(self.N):
            # Each object must have exactly one orientation
            self.model += pl.lpSum(self.d[i]) == 1, f"Orientation_{i}"

            # Define transformed dimensions based on orientation
            self.model += self.l_hat[i] == (
                self.d[i][0] * self.l[i] + self.d[i][1] * self.l[i] +
                self.d[i][2] * self.w[i] + self.d[i][3] * self.w[i] +
                self.d[i][4] * self.h[i] + self.d[i][5] * self.h[i]
            )
            self.model += self.w_hat[i] == (
                self.d[i][0] * self.w[i] + self.d[i][1] * self.h[i] +
                self.d[i][2] * self.l[i] + self.d[i][3] * self.h[i] +
                self.d[i][4] * self.l[i] + self.d[i][5] * self.w[i]
            )
            self.model += self.h_hat[i] == (
                self.d[i][0] * self.h[i] + self.d[i][1] * self.w[i] +
                self.d[i][2] * self.h[i] + self.d[i][3] * self.l[i] +
                self.d[i][4] * self.w[i] + self.d[i][5] * self.l[i]
            )

            # Object must be within container limits
            self.model += self.x[i] + self.l_hat[i] <= self.L
            self.model += self.y[i] + self.w_hat[i] <= self.W
            self.model += self.z[i] + self.h_hat[i] <= self.H

        # Pairwise non-overlapping constraints
        for i in range(self.N):
            for j in range(self.N):
                if i != j:
                    self.model += self.u[i][j] + self.s[i][j] + self.b[i][j] + self.u[j][i] + self.s[j][i] + self.b[j][i] >= 1
                    self.model += self.u[i][j] + self.u[j][i] <= 1
                    self.model += self.s[i][j] + self.s[j][i] <= 1
                    self.model += self.b[i][j] + self.b[j][i] <= 1
                    self.model += self.x[i] - self.x[j] + self.L * self.s[i][j] <= self.L - self.l_hat[i]
                    self.model += self.y[i] - self.y[j] + self.W * self.b[i][j] <= self.W - self.w_hat[i]
                    self.model += self.z[i] - self.z[j] + self.H * self.u[i][j] <= self.H - self.h_hat[i]

    def set_objective(self):
        """
        Set the objective function (Minimize weighted center positions).
        """
        self.model += pl.lpSum(
            self.wx * (self.x[i] + 0.5 * self.l_hat[i]) +
            self.wy * (self.y[i] + 0.5 * self.w_hat[i]) +
            self.wz * (self.z[i] + 0.5 * self.h_hat[i])
            for i in range(self.N)
        ), "Minimize_Center_Weights"

    def solve(self, max_solutions=10):
        """
        Solve the MILP model using PuLP with CBC and attempt to find multiple solutions.
        :param max_solutions: Maximum number of solutions to retrieve.
        """
        # Use CBC solver with `maxsol` option
        solver = pl.PULP_CBC_CMD(msg=True, options=[f"maxsol {10}"])

        # Solve the model
        self.model.solve(solver)

        # Check if optimal solution is found
        if pl.LpStatus[self.model.status] != "Optimal":
            print("No feasible solution found.")
            return []

        # Extract and return the solutions
        return self.get_solution()

    def get_solution(self):
        """
        Extract the solution from the PuLP model.
        """
        if pl.LpStatus[self.model.status] == "Optimal":
            solution = {
                'x': [self.x[i].varValue for i in range(self.N)],
                'y': [self.y[i].varValue for i in range(self.N)],
                'z': [self.z[i].varValue for i in range(self.N)],
                'l_hat': [self.l_hat[i].varValue for i in range(self.N)],
                'w_hat': [self.w_hat[i].varValue for i in range(self.N)],
                'h_hat': [self.h_hat[i].varValue for i in range(self.N)]
            }
            return solution
        else:
            raise Exception("No optimal solution found.")

# Example usage
if __name__ == "__main__":
    N = 3
    L, W, H = 10, 10, 10
    l = [2, 3, 4]
    w = [3, 2, 5]
    h = [4, 5, 3]
    wx, wy, wz = 1, 1, 1

    solver = PackingProblemSolverPuLP(N, L, W, H, l, w, h, wx, wy, wz)
    solver.add_constraints()
    solver.set_objective()
    solution = solver.solve()
    print("Solution:", solution)